# Algorithm Testing
A notebook used to compare algorithm performance.


In [1]:
### Imports
from nitools import utils 
from nitools.classifiers import PAE_RVFL, ELM, AE_ELM, PAE_ELM
from nitools.models import LeNet5, LRF_ELM, LRF_ELMplus, MiniRes
from nitools.operations import resetseed

import numpy as np
import random
import torch
import time as t

In [2]:
# Import dataset
mnist = utils.load_mnist()

mnist_in = 784
mnist_class = 10

In [3]:
def run(model, n=10000, nt=10000, data=mnist):
    X = None
    y = None 
    tX = None
    try:
        X = torch.from_numpy(data['train_X']).view((60000,1,28,28)).float()[:n]
        y = torch.from_numpy(data['train_y']).float()[:n]
        tX = torch.from_numpy(data['test_X']).view((10000,1,28,28)).float()
    except:
        X = data['train_X'].view((50000,3,32,32)).float()[:n]
        y = torch.from_numpy(data['train_y']).float()[:n]
        tX = data['test_X'].view((10000,3,32,32)).float()

    train_start = t.time()
    result = model.train(X, y)
    train_end = t.time()
    

    test_start = t.time()
    pred = model.predict(tX)[:nt]
    test_end = t.time()

    pred_arg = np.zeros(nt)

    for i in range(len(pred)):
        pred_arg[i] = np.argmax(pred[i])


    print(f"train: {train_end-train_start}")
    print(f"test: {test_end-test_start}")
    utils.evaluation_summary('lenet  - MNIST', pred_arg, data['test_y'])
    
def runos(model, b=5000, iters=1, nt=10000, data=mnist):
    X = torch.from_numpy(data['train_X']).view((60000,1,28,28)).float()
    y = torch.from_numpy(data['train_y']).float()

    train_start = t.time()
    for i in range(0,60000,b):
        if i/b==iters:
            break
        result = model.train(X[i:i+b], y[i:i+b])
    train_end = t.time()
    
    tX = torch.from_numpy(data['test_X']).view((10000,1,28,28)).float()

    test_start = t.time()
    pred = model.predict(tX)[:nt]
    test_end = t.time()

    pred_arg = np.zeros(nt)

    for i in range(len(pred)):
        pred_arg[i] = np.argmax(pred[i])


    print(f"train: {train_end-train_start}")
    print(f"test: {test_end-test_start}")
    utils.evaluation_summary('lenet  - MNIST', pred_arg, data['test_y'])

## 1. MNIST

Dataset of greyscale 28x28 images of handwritten digits. 

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*


In [ ]:
SEED = 2359487
resetseed(SEED)

model = LRF_ELM(c=1, _lambda=0, p=0.1)

run(model, n=60000)

## 2. CIFAR-10

Dataset of colour (3-dimension) 32x32 images of objects of 10 classes:

0.	airplane
1.	automobile
2.	bird
3.	cat
4.	deer
5.	dog
6.	frog
7.	horse
8.	ship
9.	truck

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*

In [4]:
# Import dataset
cifar10 = utils.load_cifar10(scaled=False)

cifar_in = 32*32*3
cifar_class = 10

In [ ]:
SEED = 22
resetseed(SEED)

model = LRF_ELMplus(in_channels=3)

run(model, n=5000, data=cifar10)

torch.Size([5000, 3072])
